In [1]:
import os
from PIL import Image

In [2]:
# Image resizing parameters
img_rows, img_cols = 256, 256
isolated_train_path = 'D:/CXR8/ChestX-ray'
isolated_test_path = 'D:/CXR8/ChestX-ray Test'

In [3]:
# Preprocess images
for root, _, filenames in os.walk(isolated_train_path):
    for filename in filenames:
        file_path = os.path.join(root, filename)
        img = Image.open(file_path)
        img = img.resize((img_rows, img_cols))
        img = img.convert('RGB')
        img.save(file_path)

for root, _, filenames in os.walk(isolated_test_path):
    for filename in filenames:
        file_path = os.path.join(root, filename)
        img = Image.open(file_path)
        img = img.resize((img_rows, img_cols))
        img = img.convert('RGB')
        img.save(file_path)

In [4]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

In [5]:
# Paths for training and validation data
train_data_dir = isolated_train_path 
validation_data_dir = isolated_test_path

In [6]:
# Image data parameters
img_width, img_height, channel = img_rows, img_cols, 3
batch_size = 32

In [7]:
# Data augmentation and normalization
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.1  # 10% for validation
)

In [8]:
# Training and validation data generators
train_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    shuffle=True,
    class_mode='categorical',
    subset='training'
)

Found 9450 images belonging to 15 classes.


In [9]:
validation_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    shuffle=True,
    class_mode='categorical',
    subset='validation'
)

Found 1050 images belonging to 15 classes.


In [10]:
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, ReLU, Dense, Dropout, GlobalAvgPool2D, Concatenate, AvgPool2D

In [11]:
# Model building functions
def batch_normalization_relu_conv(x, filters, kernel_size):
    x = Conv2D(filters=filters, kernel_size=kernel_size, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    return x

def dense_block(tensor, k, reps):
    for _ in range(reps):
        x = batch_normalization_relu_conv(tensor, filters=4*k, kernel_size=1)
        x = batch_normalization_relu_conv(x, filters=k, kernel_size=3)
        tensor = Concatenate()([tensor, x])
    return tensor

def transition_layer(x, theta):
    f = int(tensorflow.keras.backend.int_shape(x)[-1] * theta)
    x = batch_normalization_relu_conv(x, filters=f, kernel_size=1)
    x = AvgPool2D(pool_size=2, strides=2, padding='same')(x)
    return x

In [13]:
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, ReLU, Dense, Dropout, GlobalAvgPool2D, Concatenate, AvgPool2D
import tensorflow as tf


In [15]:
# Model building functions
def batch_normalization_relu_conv(x, filters, kernel_size):
    x = Conv2D(filters=filters, kernel_size=kernel_size, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    return x

def dense_block(tensor, k, reps):
    for _ in range(reps):
        x = batch_normalization_relu_conv(tensor, filters=4*k, kernel_size=1)
        x = batch_normalization_relu_conv(x, filters=k, kernel_size=3)
        tensor = Concatenate()([tensor, x])
    return tensor       

def transition_layer(x, theta):
    f = int(tf.keras.backend.int_shape(x)[-1] * theta)
    x = batch_normalization_relu_conv(x, filters=f, kernel_size=1)
    x = AvgPool2D(pool_size=2, strides=2, padding='same')(x)
    return x

In [16]:
# Model architecture
k = 12
theta = 0.5
repetitions = [10, 14, 16]
input_layer = Input(shape=(img_width, img_height, channel))

x = Conv2D(2*k, 7, strides=2, padding='same')(input_layer)
x = Dropout(0.2)(x)
x = MaxPooling2D(pool_size=3, strides=2, padding='same')(x)

for reps in repetitions:
    x = dense_block(x, k, reps)
    x = transition_layer(x, theta)

x = GlobalAvgPool2D()(x)
output_layer = Dense(15, activation='softmax')(x)

model = Model(input_layer, output_layer)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_42 (Conv2D)             (None, 128, 128, 24  3552        ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 dropout_2 (Dropout)            (None, 128, 128, 24  0           ['conv2d_42[0][0]']              
                                )                                                             

In [20]:

# Model compilation
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [21]:
# Model training
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=100,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)


Epoch 1/100
295/295 [==============================] - 171s 553ms/step - loss: 2.6139 - accuracy: 0.1377 - val_loss: 40.1036 - val_accuracy: 0.0654
Epoch 2/100
295/295 [==============================] - 138s 466ms/step - loss: 2.5323 - accuracy: 0.1648 - val_loss: 3.3005 - val_accuracy: 0.0938
Epoch 3/100
295/295 [==============================] - 137s 464ms/step - loss: 2.4997 - accuracy: 0.1714 - val_loss: 2.8457 - val_accuracy: 0.0752
Epoch 4/100
295/295 [==============================] - 137s 463ms/step - loss: 2.4766 - accuracy: 0.1767 - val_loss: 2.8055 - val_accuracy: 0.0957
Epoch 5/100
295/295 [==============================] - 136s 461ms/step - loss: 2.4574 - accuracy: 0.1806 - val_loss: 2.7754 - val_accuracy: 0.0869
Epoch 6/100
295/295 [==============================] - 135s 457ms/step - loss: 2.4502 - accuracy: 0.1887 - val_loss: 3.2041 - val_accuracy: 0.0645
Epoch 7/100
295/295 [==============================] - 134s 452ms/step - loss: 2.4298 - accuracy: 0.1936 - val_loss: 

KeyboardInterrupt: 

In [ ]:
# Plot training & validation accuracy values
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], 'o-', label='train accuracy')
plt.plot(history.history['val_accuracy'], 'o-', label='validation accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Model Accuracy')
plt.grid(True)
plt.legend(loc='lower right')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], 'o-', label='train loss')
plt.plot(history.history['val_loss'], 'o-', label='validation loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Model Loss')
plt.grid(True)
plt.legend(loc='upper right')

plt.show()

# Model evaluation
Y_pred = model.predict(validation_generator, validation_generator.samples // batch_size)
y_pred = np.argmax(Y_pred, axis=1)
y_true = validation_generator.classes



In [ ]:
# Classification report and confusion matrix
from sklearn.metrics import classification_report, confusion_matrix

target_names = list(validation_generator.class_indices.keys())
print(classification_report(y_true, y_pred, target_names=target_names, digits=3))
